### Imports and loading 

In [ ]:
import sys
from pathlib import Path

# Add project root to Python path
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

import asyncio
import dspy
from dspy import GEPA

from prompt_optimization.config import (
    LMConfig,
)
from prompt_optimization.dataset_loaders import (
    load_aimo_datasets,
    load_frames_dataset,
    load_seal0_dataset,
    load_simpleqa_verified_dataset,
)
from prompt_optimization.judge import ComponentJudge
from prompt_optimization.metrics import MetricWithFeedback
from prompt_optimization.optimizer import create_optimizer
from prompt_optimization.prompts import (
    AGGREGATOR_PROMPT,
    ATOMIZER_DEMOS,
    ATOMIZER_PROMPT,
    GRADER_PROMPT,
    PLANNER_DEMOS,
    PLANNER_PROMPT,
)
from roma_dspy.config import load_config
from roma_dspy.core.engine.solve import RecursiveSolver
from roma_dspy.core.modules.recursive_solver import RecursiveSolverModule
from roma_dspy.utils import AsyncParallelExecutor

dspy.settings.provide_traceback = True  # optional but mirrors the old notebook
import os
os.chdir(project_root)  # Change to project root for config loading

opt_cfg = load_config(profile="test")

### Config LLMS

In [2]:
# Batch the knobs you used to tweak in the notebook.
opt_cfg.train_size = 32
opt_cfg.val_size = 12
opt_cfg.test_size = 12
opt_cfg.dataset_seed = 42
opt_cfg.max_metric_calls = 225
opt_cfg.max_depth = 1
opt_cfg.enable_logging = False

In [3]:
#Add few-shot examples + prompts
opt_cfg.agents.atomizer.signature_instructions = ATOMIZER_PROMPT
opt_cfg.agents.planner.signature_instructions = PLANNER_PROMPT
opt_cfg.agents.aggregator.signature_instructions = AGGREGATOR_PROMPT

### Init solvers and what not

In [4]:
solver = RecursiveSolver(
    config=opt_cfg,
    max_depth=opt_cfg.max_depth,
    enable_logging=opt_cfg.enable_logging,
    enable_checkpoints=False,
)
solver_module = RecursiveSolverModule(solver=solver)

2025-10-20 22:42:56.788 | DEBUG    | roma_dspy.core.factory.agent_factory:_resolve_signature:119 - Using default signature for atomizer
2025-10-20 22:42:56.789 | INFO     | roma_dspy.core.factory.agent_factory:create_agent:91 - Created atomizer agent (task_type=default, signature=default)
2025-10-20 22:42:56.790 | DEBUG    | roma_dspy.core.registry.agent_registry:register_agent:180 - Registered atomizer instance #1 (task_type=default)
2025-10-20 22:42:56.791 | DEBUG    | roma_dspy.core.factory.agent_factory:_resolve_signature:119 - Using default signature for planner
2025-10-20 22:42:56.793 | INFO     | roma_dspy.core.factory.agent_factory:create_agent:91 - Created planner agent (task_type=default, signature=default)
2025-10-20 22:42:56.794 | DEBUG    | roma_dspy.core.registry.agent_registry:register_agent:180 - Registered planner instance #2 (task_type=default)
2025-10-20 22:42:56.795 | DEBUG    | roma_dspy.core.factory.agent_factory:_resolve_signature:119 - Using default signature fo

In [5]:
from prompt_optimization.metrics.search_metric import SearchMetric
from prompt_optimization.prompts.grader_prompts.search_grader_prompt import SEARCH_GRADER_PROMPT


scorer_lm = LMConfig("openrouter/google/gemini-2.5-flash-lite-preview-09-2025", temperature=0.3, max_tokens=128000, cache=True)
component_lm = LMConfig("openrouter/anthropic/claude-sonnet-4.5", temperature=0.45, max_tokens=128000, cache=True)


In [6]:
search_metric_scorer = SearchMetric(prompt=SEARCH_GRADER_PROMPT, lm_config=scorer_lm)
component_metric_scorer = ComponentJudge(prompt=GRADER_PROMPT, lm_config=component_lm)
metric = MetricWithFeedback(judge = component_metric_scorer, scoring_metric = search_metric_scorer)

In [7]:

train_set, val_set, test_set = load_frames_dataset(
    train_size=opt_cfg.train_size,
    val_size=opt_cfg.val_size,
    test_size=opt_cfg.test_size,
    seed=opt_cfg.dataset_seed,
)

### Perform an eval on the val set

In [ ]:
# executor = AsyncParallelExecutor(max_concurrency=4)

# results = await executor.execute_batch(solver_module, test_set)

In [ ]:
# print(results)

### Prompt tuning stuff

In [ ]:
optimizer = GEPA(
    metric=metric,
    # auto="light",
    component_selector="round_robin",
    max_metric_calls=48,
    add_format_failure_as_feedback=True,
    num_threads=8,
    track_stats=True,
    log_dir="logs/frames_test",
    # use_wandb=True,
    # wandb_init_kwargs={"project": "aime_test"},
    reflection_minibatch_size=8,
    reflection_lm=dspy.LM(model="openrouter/anthropic/claude-sonnet-4.5", temperature=.75, max_tokens=128000)
)

In [ ]:
optimized_program = optimizer.compile(
    solver_module,
    trainset=train_set[:8],
    valset=val_set,
)

In [ ]:
dir(optimized_program.named_predictors()[1][1])